In [1]:
import os
import time

from selenium import webdriver
from selenium.webdriver.common.by import By

chromedriver_path = "C:\\Users\\Araf\\Desktop\\Script\\chromedriver_win32"
os.environ['PATH'] += chromedriver_path

In [2]:
driver = webdriver.Chrome()
driver.implicitly_wait(40)

driver.get("https://atcoder.jp")
# do login manually

In [16]:
# there are 9 pages of contests
page = 9
driver.get(f"https://atcoder.jp/contests/archive?page={page}")
time.sleep(7)
contests = driver.find_elements(By.XPATH, "//div/div/div/div[3]/div[2]/div/table/tbody/tr") # all contest links

contest_links = []
for _ in contests:
    contest_links.append(str(_.find_element(By.CSS_SELECTOR, 'a[href*="/contests"]').get_attribute("href")))

In [17]:
lt = 0
rt = len(contest_links)-1

for i in range(lt, rt+1): # for each contest
    
    print(f"parsing: {i} of {rt}")
    
    driver.get(contest_links[i]+"/submissions/me") # go to my submission page
    time.sleep(6)
    
    all_text = driver.find_element(By.TAG_NAME, "body").text
    if "No Submissions" in all_text:
        continue
            
    rows = driver.find_elements(By.XPATH, "//div/table/tbody/tr") # for each submission
    code_pages = []
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols)<10:
            continue
        code_pages.append(cols[9].find_element(By.TAG_NAME, "a").get_attribute("href"))
    
    for j in range(len(code_pages)):
        driver.get(code_pages[j])
        time.sleep(6)
        
        detail = driver.find_elements(By.XPATH, "//body/div[3]/div/div/div[2]/div[3]/table/tbody/tr/td")
        verdict = detail[6].text.strip()
        created = detail[0].text.strip()
        title = detail[1].text.strip()
        p_link = detail[1].find_element(By.CSS_SELECTOR, "a[href*='tasks']").get_attribute("href")
        language = detail[3].text.strip()
        run_time = detail[7].text.strip()
        mem_used = detail[8].text.strip()
        
        code = driver.find_element(By.CSS_SELECTOR, ".linenums").text
        
        table_data=[
            [f'*  @author: kzvd4729', f'created: {created}'],
            [f'*  solution_verdict: {verdict}', f'language: {language}'],
            [f'*  run_time: {run_time}', f'memory_used: {mem_used}']
        ]
        printer = '/****************************************************************************************\n'
        for row in table_data:
            printer+=("{: <60} {: <50}".format(*row))+'\n'
        printer += f"*  problem: {p_link}\n"
        printer +='****************************************************************************************/\n'
        
        printer += code
        
        if len(printer)<590:
            print("got one bad")
            exit()
        
        for c in "\/:*?\"<>|.": # folder name can't have these character
            title = title.replace(c, 'x')
            
        current = f"atcoder/{title}"
        if not os.path.exists(current):
            os.mkdir(current)
        
        current = current+f'/{verdict}'
        for k in range(1, 1000):
            if (k==1):
                now = current+'.cpp'
            else:
                now = current+f' ({k}).cpp'

            if not os.path.exists(now):
                with open(now, 'w') as f:
                    f.write(printer)
                f.close()

                break
        print(f"submission:  {j}/{len(code_pages)-1}")
        

parsing: 0 of 9
parsing: 1 of 9
parsing: 2 of 9
parsing: 3 of 9
submission:  0/2
submission:  1/2
submission:  2/2
parsing: 4 of 9
parsing: 5 of 9
parsing: 6 of 9
parsing: 7 of 9
parsing: 8 of 9
parsing: 9 of 9
